In [1]:
!pip install ../input/kerasapplications/keras-team-keras-applications-3b180cb -f ./ --no-index >/dev/null
!pip install ../input/effnet/efficientnet-master -f ./ --no-index >/dev/null

In [2]:
import os
import cv2
import pydicom
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm 
from sklearn.model_selection import train_test_split, KFold

import efficientnet.tfkeras as efn
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import tensorflow.keras.activations as activations
from tensorflow_addons.optimizers import AdamW
from tensorflow.keras import Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.layers import (
    Dense, Dropout, Activation, Flatten, Input, BatchNormalization, GlobalAveragePooling2D, Add,
    Conv2D, AveragePooling2D, LeakyReLU, Concatenate, DepthwiseConv2D, Maximum, Layer)

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


# Training Parameters

- `EPOCHS`: number of epochs to train for in each fold
- `BATCH_SIZE`: batch size of images during training
- `NFOLD`: number of folds in K-fold cross-validation (CV)
- `LR_STRATEGY`: options are [ReduceLROnPlateau](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau) or [CosineDecayRestarts](https://www.tensorflow.org/api_docs/python/tf/keras/experimental/CosineDecayRestarts)
- `START_LR`: the initial learning rate 
- `SAVE_BEST`: default is True to save best weights on validation loss
- `MODEL_CLASS`: the class of model. E.g. "b1" for EfficientNet-B1
- `IM_SIZE`: Image size to use (shape would be (IM_SIZE,IM_SIZE,1))
- `FIRST_DECAY_STEPS` passed to [CosineDecayRestarts](https://www.tensorflow.org/api_docs/python/tf/keras/experimental/CosineDecayRestarts) learning rate scheduler
- `WD` Weight decay passed to the AdamW optimizer
- `PATIENCE` number  of epochs to wait without improvement on val_loss
- `TRAIN_STEPS` number of steps per epoch in training
- `VAL_STEPS` number of steps per epoch in validation

In [3]:
CROSS_VALIDATION = False
EPOCHS      = 200
BATCH_SIZE  = 8
NFOLD       = 3
LR_STRATEGY = 'CosineDecayRestarts' #options ['ReduceLROnPlateau','CosineDecayRestarts']
START_LR    = 1e-2
SAVE_BEST   = True
MODEL_CLASS = 'b5'
IM_SIZE     = 512
FIRST_DECAY_STEPS = 1e1
WD          = 1e-2
PATIENCE    = 60
TRAIN_STEPS = 60
VAL_STEPS   = 16

In [4]:
class FReLU(Layer):
    def __init__(self, kernel_size=3):
        super(FReLU, self).__init__()
        self.conv = DepthwiseConv2D(kernel_size=kernel_size, padding="same")
        self.bn   = BatchNormalization()

    def call(self, inputs):
        x1 = self.conv(inputs)
        x1 = self.bn(x1)
        x  = K.maximum(inputs, x1)
        return x

    def compute_output_shape(self, input_shape):
        return input_shape
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'conv': self.conv,
            'bn': self.bn
        })
        return config
    
def get_efficientnet(model, shape):
    models_dict = {
        'b0': efn.EfficientNetB0(input_shape=shape,weights=None,include_top=False),
        'b1': efn.EfficientNetB1(input_shape=shape,weights=None,include_top=False),
        'b2': efn.EfficientNetB2(input_shape=shape,weights=None,include_top=False),
        'b3': efn.EfficientNetB3(input_shape=shape,weights=None,include_top=False),
        'b4': efn.EfficientNetB4(input_shape=shape,weights=None,include_top=False),
        'b5': efn.EfficientNetB5(input_shape=shape,weights=None,include_top=False),
        'b6': efn.EfficientNetB6(input_shape=shape,weights=None,include_top=False),
        'b7': efn.EfficientNetB7(input_shape=shape,weights=None,include_top=False)
    }
    return models_dict[model]

def redifine_efn_model(model):
    for i, layer in enumerate(model.layers):
        if i==0:
            input = layer.input
            x     = input
            seblock_on = False
            block_prev = ""
            continue
            
        block_curt = layer.name[:layer.name.find("_")]
        if block_prev != block_curt:
            input_block = x
        if   -1 < layer.name.find("activation"):
            x = FReLU()(x)
        elif -1 < layer.name.find("se"):
            if -1 < layer.name.find("excite"):
                x  = layer([x, x_seblock])
                seblock_on = False
            elif -1 < layer.name.find("reduce"):
                layer.activation = activations.linear
                x_seblock = layer(x_seblock)
                x_seblock = FReLU()(x_seblock)
            elif not seblock_on:
                seblock_on = True 
                x_seblock  = layer(x)
            else:
                x_seblock  = layer(x_seblock)
        elif -1 < layer.name.find("add"):
            x = layer([x, input_block])
        else:
            x = layer(x)      
        block_prev = block_curt
    return Model(input, x)

def build_model(shape=(IM_SIZE, IM_SIZE, 1), model_class=None):
    inp  = Input(shape=shape)
    base = get_efficientnet(model_class, shape)
    base = redifine_efn_model(base)
    x  = base(inp)
    x  = GlobalAveragePooling2D()(x)
    inp2 = Input(shape=(4,))
    x2 = tf.keras.layers.GaussianNoise(0.2)(inp2)
    x  = Concatenate()([x, x2]) 
    x  = Dropout(0.5)(x)
    # Explicit f32 because mixed precision 
    x  = Dense(1, dtype='float32')(x)
    model = Model([inp, inp2] , x)
    return model

In [5]:
#plt.imshow(get_img("../input/osic-pulmonary-fibrosis-progression/train/ID00014637202177757139317/10.dcm"), cmap="gray")
#plt.imshow(get_img("../input/osic-pulmonary-fibrosis-progression/train/ID00248637202266698862378/4.dcm"), cmap="gray")

In [6]:
def get_img(path):
    def rescale(img, ri, rs):
        return (img - ri) / (rs * 1000)
    d    = pydicom.dcmread(path)
    img  = d.pixel_array
    img[img <= -1000] = 0
    img  = img.astype(np.int32)
    W, H = img.shape
    if W == H:
        img = rescale(img, d.RescaleIntercept, d.RescaleSlope)
        if W == IM_SIZE:
            return img
        else:
            return cv2.resize(img, (IM_SIZE, IM_SIZE))
    iimg   = Image.fromarray(img, mode="I")
    left   = (img.shape[1]-512)/2
    right  = (img.shape[1]+512)/2
    top    = (img.shape[0]-512)/2
    bottom = (img.shape[0]+512)/2
    iimg   = iimg.crop((left, top, right, bottom))
    iimg   = iimg.resize((IM_SIZE, IM_SIZE), resample=Image.LANCZOS)
    return rescale(np.array(iimg), d.RescaleIntercept, d.RescaleSlope)

class IGenerator(Sequence):
    BAD_ID = ['ID00011637202177653955184', 'ID00052637202186188008618']
    def __init__(self, keys, a, tab, batch_size=BATCH_SIZE):
        self.keys = [k for k in keys if k not in self.BAD_ID]
        self.a    = a
        self.tab  = tab
        self.batch_size = batch_size
        
        self.train_data = {}
        for p in train.Patient.values:
            self.train_data[p] = os.listdir(f'../input/osic-pulmonary-fibrosis-progression/train/{p}/')
    
    def __len__(self):
        return 1000
    
    def __getitem__(self, idx):
        x, a, tab = [], [], []
        keys = np.random.choice(self.keys, size = self.batch_size)
        for k in keys:
            try:
                i   = np.random.choice(self.train_data[k], size=1)[0]
                img = get_img(f'../input/osic-pulmonary-fibrosis-progression/train/{k}/{i}')
                x.append(img)
                a.append(self.a[k])
                tab.append(self.tab[k])
            except:
                print(k, i)
       
        x, a, tab = np.array(x), np.array(a), np.array(tab)
        x = np.expand_dims(x, axis=-1)
        return [x, tab] , a

In [7]:
def preparation():
    # train data
    train = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
    train.drop_duplicates(keep=False, inplace=True, subset=['Patient','Weeks'])
    return train

def make_base(data):
    data['min_week'] = data['Weeks']
    data['min_week'] = data.groupby('Patient')['min_week'].transform('min')
    
    base = data.loc[data.Weeks == data.min_week][['Patient','FVC',"Percent"]].copy()
    base.columns = ['Patient','min_FVC','min_Percent']
    base['nb']   = 1
    base['nb']   = base.groupby('Patient')['nb'].transform('cumsum')
    base = base[base.nb==1]
    base.drop('nb', axis=1, inplace=True)
    return base

def normalize(d, c, c_new):
    d = d.copy()
    if c not in d.columns: return d
    c_min = d[c].min()
    c_max = d[c].max()
    print(c_min, c_max)
    d[c_new] = (d[c] - d[c].min()) / (d[c].max() - d[c].min())
    return d

def get_tab(df):
    vector = [(df.Age.values[0] - 30) / 30] 
    #vector.append(df.min_Percent.values[0])
    if df.Sex.values[0] == 'male':
        vector.append(0)
    else:
        vector.append(1)    
    if   df.SmokingStatus.values[0] == 'Never smoked':
        vector.extend([0,0])
    elif df.SmokingStatus.values[0] == 'Ex-smoker':
        vector.extend([1,1])
    elif df.SmokingStatus.values[0] == 'Currently smokes':
        vector.extend([0,1])
    else:
        vector.extend([1,0])
    return np.array(vector) 

# Training

In [8]:
train = preparation()
train = train.merge(make_base(train), on='Patient', how='left')
train = normalize(train, "min_Percent", "min_Percent")
train.head()

43.352279059400395 153.012912482066


,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,min_week,min_FVC,min_Percent
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,-4,2315,0.135886
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,-4,2315,0.135886
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,-4,2315,0.135886
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,-4,2315,0.135886
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,-4,2315,0.135886


In [9]:
A = {}; TAB = {}; P = [] 
for i, p in enumerate(train.Patient.unique()):
    sub    = train.loc[train.Patient == p, :] 
    fvc    = sub.FVC.values
    weeks  = sub.Weeks.values
    c      = np.vstack([weeks, np.ones(len(weeks))]).T
    # Least squares(c * a = fvc)
    a, b   = np.linalg.lstsq(c, fvc, rcond=-1)[0]
    A[p]   = a
    TAB[p] = get_tab(sub)
    P.append(p)

In [10]:
strategy = tf.distribute.MirroredStrategy()

# Enabling mixed precision
config  = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
policy  = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

# Callbacks
er = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=1e-3,
    patience=PATIENCE,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=True)

rlp = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5,
    patience=5, 
    verbose=1, 
    min_lr=1e-8)
# Cosine decay restarts, works well with AdamW and convergences faster
lr_schedule = tf.keras.experimental.CosineDecayRestarts(START_LR,FIRST_DECAY_STEPS)
lrs_cdr     = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

# Start training
P    = np.array(P)
subs = []
folds_history = []
with strategy.scope():

    if CROSS_VALIDATION:
        kf = KFold(n_splits=NFOLD, shuffle=False)
        for fold, (tr_idx, val_idx) in enumerate(kf.split(P)):
            print('#####################')
            print('####### Fold %i ######'%fold)
            print('#####################')

            print('Training...')

            cpt = tf.keras.callbacks.ModelCheckpoint(
                filepath='%s_fold-%i.h5'%(MODEL_CLASS, fold),
                monitor='val_loss', 
                verbose=1, 
                save_best_only=SAVE_BEST,
                mode='auto')

            callbacks = [cpt,er]
            if   LR_STRATEGY=='ReduceLROnPlateau':   callbacks.append(rlp)
            elif LR_STRATEGY=='CosineDecayRestarts': callbacks.append(lrs_cdr)
            optimizer = tfa.optimizers.AdamW(weight_decay=WD)
            #Loss scaling for GPU
            optimizer = mixed_precision.LossScaleOptimizer(optimizer, loss_scale='dynamic')

            model   = build_model(model_class=MODEL_CLASS)
            model.compile(optimizer=optimizer, loss="mae") 
            history = model.fit_generator(IGenerator(keys=P[tr_idx],
                                                     a=A,
                                                     tab=TAB),
                                          steps_per_epoch=TRAIN_STEPS,
                                          validation_data=IGenerator(keys=P[val_idx],
                                                                     a=A,
                                                                     tab=TAB),
                                          validation_steps=VAL_STEPS, 
                                          callbacks=callbacks, 
                                          epochs=EPOCHS)
            folds_history.append(history.history)
    else:
        tr_idx, val_idx = train_test_split(P, test_size=0.2)
        
        cpt = tf.keras.callbacks.ModelCheckpoint(
            filepath='%s_epoch%s.h5'%(MODEL_CLASS, EPOCHS),
            monitor='val_loss', 
            verbose=1, 
            save_best_only=SAVE_BEST,
            mode='auto')

        callbacks = [cpt,er]
        if   LR_STRATEGY=='ReduceLROnPlateau':   callbacks.append(rlp)
        elif LR_STRATEGY=='CosineDecayRestarts': callbacks.append(lrs_cdr)
        optimizer = tfa.optimizers.AdamW(weight_decay=WD)
        #Loss scaling for GPU
        optimizer = mixed_precision.LossScaleOptimizer(optimizer, loss_scale='dynamic')

        model   = build_model(model_class=MODEL_CLASS)
        model.compile(optimizer=optimizer, loss="mae") 
        history = model.fit_generator(IGenerator(keys=tr_idx,
                                                 a=A,
                                                 tab=TAB),
                                      steps_per_epoch=TRAIN_STEPS,
                                      validation_data=IGenerator(keys=val_idx,
                                                                 a=A,
                                                                 tab=TAB),
                                      validation_steps=VAL_STEPS, 
                                      callbacks=callbacks, 
                                      epochs=EPOCHS)
        folds_history.append(history.history)
        
    print('Training done!')

Epoch 1/200
60/60 [==============================] - ETA: 0s - loss: 4.9146
Epoch 00001: val_loss improved from inf to 4.51741, saving model to b5_epoch200.h5
60/60 [==============================] - 73s 1s/step - loss: 4.9146 - val_loss: 4.5174
Epoch 2/200
60/60 [==============================] - ETA: 0s - loss: 4.7316
Epoch 00002: val_loss did not improve from 4.51741
60/60 [==============================] - 60s 1s/step - loss: 4.7316 - val_loss: 4.6977
Epoch 3/200
60/60 [==============================] - ETA: 0s - loss: 4.5909
Epoch 00003: val_loss did not improve from 4.51741
60/60 [==============================] - 60s 1s/step - loss: 4.5909 - val_loss: 4.7810
Epoch 4/200
60/60 [==============================] - ETA: 0s - loss: 3.9845
Epoch 00004: val_loss did not improve from 4.51741
60/60 [==============================] - 60s 994ms/step - loss: 3.9845 - val_loss: 4.5794
Epoch 5/200
60/60 [==============================] - ETA: 0s - loss: 4.2857
Epoch 00005: val_loss improved fr

# CV Evaluation

In [11]:
if SAVE_BEST:
    mean_val_loss = np.mean([np.min(h['val_loss']) for h in folds_history])
else:
    mean_val_loss = np.mean([h['val_loss'][-1] for h in folds_history])
print('Our mean CV MAE is: ' + str(mean_val_loss))

Our mean CV MAE is: 3.6691577434539795
